# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [11]:
# import libraries
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re

import nltk
nltk.download(['punkt','wordnet','stopwords','words'])
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, words

[nltk_data] Downloading package punkt to /Users/paul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/paul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/paul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /Users/paul/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [12]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db/')
df = pd.read_sql_table('MessageToCategories',con=engine)
X = df['message'].values 
Y = df.iloc[:,4:]
#X = df.message.values
#Y = df[df.columns[4:]].values
classes = Y.columns
classes

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [15]:
X = df['message'].values 
Y = df.iloc[:,4:]
#X = df.message.values
#Y = df[df.columns[4:]].values
classes = Y.columns
classes

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [16]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df.iloc[219,5]

0

In [18]:
X[:5]

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight'],
      dtype=object)

In [19]:
Y[:10]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
len(X)

26028

In [21]:
len(Y)

26028

### 2. Write a tokenization function to process your text data

In [22]:
def tokenize(text):
    #wrds = set(words.words())
    text = re.sub(r"[^a-zA-Z]"," ",text).lower()
    tokens = word_tokenize(text)
    #tokens = [t for t in tokens if t not in stopwords.words('english')]
    #tokens = [t for t in tokens if t in wrds]
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [23]:
for message in X[:10]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 

Information about the National Palace-
['information', 'about', 'the', 'national', 'palace'] 

Storm at sacred heart of jesus
['storm', 'at', 'sacred', 'heart', 'of', 'jesus'] 

Please, we need tents and water. We are in Silo, Thank you!
['plea

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline

In [15]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', OneVsRestClassifier(SVC(class_weight='balanced',max_iter=20000)))
                ])

#MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier()))

#pipeline = Pipeline([
#                #('vect', CountVectorizer(tokenizer=tokenize)),
#                ('tfidf', HashingVectorizer(tokenizer=tokenize)),
#                ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced'),n_jobs=1))
#                ])

#pipeline = Pipeline([
#               ('vect', CountVectorizer(ngram_range=(1,3), max_df=0.50 ) ),
#               ('tfidf', TfidfTransformer() ),
#               ('clf', OneVsRestClassifier(SGDClassifier(loss='modified_huber', penalty='elasticnet',
#                                          alpha=1e-4, n_iter=5, random_state=42,
#                                        shuffle=True, n_jobs=-1) )),
#               ])

In [13]:
#vect = CountVectorizer(tokenizer=tokenize)
#X_train = vect.fit_transform(X_train)

In [12]:
#dict = vect.vocabulary_
#sorted(dict.items(), key=lambda kv: kv[1], reverse=True)

In [42]:
'''sorted(dict.items())
new_list = {}
words = set(nltk.corpus.words.words())
for (key,value) in dict.items():
    if key in words:
        new_list.update([(key,value)])
len(new_list)'''

11295

In [13]:
#dict

In [14]:
#sorted(dict.items(), key=lambda kv: kv[1])

In [60]:
#tfidf = TfidfTransformer()
#X_train_tfidf = tfidf.fit_transform(X_train)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3,random_state=19,shuffle=True)
pipeline.fit(X_train,Y_train)

/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...lity=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
          n_jobs=None))])

In [28]:
#clf = RandomForestClassifier()
#model = MultiOutputClassifier(clf)
#model.fit(X_train_tfidf,Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
Y_pred = pipeline.predict(X_test)

In [121]:
Y_pred.shape

(7855, 36)

In [89]:
#X_test_counts = vect.transform(X_test)
#X_test_tfidf = tfidf.transform(X_test_counts)
#Y_pred = model.predict(X_test_tfidf)

In [122]:
overall_accuracy = (Y_pred == Y_test).mean().mean()
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 93.43% 



In [75]:
#print(classification_report(Y_test[1],Y_pred[1],labels=36, target_names=classes))
#print(classification_report(Y_test.iloc[:,0:].values, np.array([x[0:] for x in Y_pred]), target_names=classes))
#print(classification_report(Y_test, Y_pred))
print(classification_report(Y_pred, Y_test.values, target_names=classes))

In [123]:
Y_pred_df = pd.DataFrame(Y_pred, columns = Y_test.columns)
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(Y_test[column],Y_pred_df[column]))

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.56      0.70      0.62      1778
           1       0.90      0.84      0.87      6077

   micro avg       0.81      0.81      0.81      7855
   macro avg       0.73      0.77      0.75      7855
weighted avg       0.83      0.81      0.81      7855

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.95      0.90      0.92      6500
           1       0.61      0.75      0.67      1355

   micro avg       0.87      0.87      0.87      7855
   macro avg       0.78      0.82      0.80      7855
weighted avg       0.89      0.87      0.88      7855

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7818
           1     

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      7449
           1       0.46      0.59      0.51       406

   micro avg       0.94      0.94      0.94      7855
   macro avg       0.72      0.78      0.74      7855
weighted avg       0.95      0.94      0.95      7855

------------------------------------------------------

FEATURE: electricity

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7716
           1       0.38      0.47      0.42       139

   micro avg       0.98      0.98      0.98      7855
   macro avg       0.69      0.73      0.70      7855
weighted avg       0.98      0.98      0.98      7855

------------------------------------------------------

FEATURE: tools

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7803
           1       0.11      0.04      0.06        52

   micro avg       0.99      0.9

In [101]:
Y_test.iloc[:,0:].values.shape

(8641, 36)

In [102]:
np.array([x[0:] for x in Y_pred]).shape

(8641, 36)

In [124]:
print("\t\t Accuracy Score: ")
print("\t\t ---------------")
for i in range(Y_test.shape[1]):
    print('%25s : %.2f' %(classes[i], accuracy_score(Y_test.values[:,i], Y_pred[:,i])))
print("\t\t ---------------")    
print('\t Overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

		 Accuracy Score: 
		 ---------------
                  related : 0.81
                  request : 0.87
                    offer : 0.99
              aid_related : 0.76
             medical_help : 0.89
         medical_products : 0.93
        search_and_rescue : 0.95
                 security : 0.97
                 military : 0.97
              child_alone : 1.00
                    water : 0.96
                     food : 0.94
                  shelter : 0.93
                 clothing : 0.99
                    money : 0.97
           missing_people : 0.98
                 refugees : 0.96
                    death : 0.96
                other_aid : 0.80
   infrastructure_related : 0.88
                transport : 0.93
                buildings : 0.94
              electricity : 0.98
                    tools : 0.99
                hospitals : 0.98
                    shops : 0.99
              aid_centers : 0.98
     other_infrastructure : 0.92
          weather_related : 0.87
    

### 6. Improve your model
Use grid search to find better parameters. 

In [125]:
from sklearn.model_selection import GridSearchCV

In [143]:
parameters = {'vect__ngram_range':[(1,1),(1,2),(1,3)],
              'clf__estimator__C':[0.75,0.9,1.0]
              }

cv = GridSearchCV(pipeline,param_grid=parameters)
cv

#param_grid = dict(reduce_dim=[None, PCA(5), PCA(10)],
#...                   clf=[SVC(), LogisticRegression()],
#...                   clf__C=[0.1, 10, 100])
#>>> grid_search = GridSearchCV(pipe, param_grid=param_grid)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'vect__ngram_range': [(1, 1), (1, 2), (1, 3)], 'clf__estimator__penalty': ['l1', 'l2'], 'clf__estimator__dual': [False], 'clf__estimator__C': [0.6, 0.7, 0.75, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [144]:
cv.fit(X_train,Y_train)

/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/anaconda2/envs/

/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/anaconda2/envs/

/anaconda2/envs/base_env_py3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'vect__ngram_range': [(1, 1), (1, 2), (1, 3)], 'clf__estimator__penalty': ['l1', 'l2'], 'clf__estimator__dual': [False], 'clf__estimator__C': [0.6, 0.7, 0.75, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [145]:
cv.best_params_

{'clf__estimator__C': 0.9,
 'clf__estimator__dual': False,
 'clf__estimator__penalty': 'l2',
 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [146]:
Y_pred = cv.predict(X_test)

In [151]:
# print classification report
print(classification_report(Y_pred, Y_test.values, target_names=classes))
# print raw accuracy score 
print('Accuracy Score: {}'.format(np.mean(Y_test.values == Y_pred)))
overall_accuracy = (Y_test == Y_pred).mean().mean()
print('Average Overall Accuracy : {0:.2f}% \n'.format(overall_accuracy*100))

                        precision    recall  f1-score   support

               related       0.90      0.89      0.89      6149
               request       0.74      0.68      0.71      1467
                 offer       0.03      0.50      0.05         2
           aid_related       0.79      0.70      0.74      3686
          medical_help       0.47      0.53      0.50       570
      medical_products       0.42      0.51      0.46       305
     search_and_rescue       0.25      0.55      0.34       103
              security       0.04      0.26      0.07        23
              military       0.48      0.52      0.50       209
           child_alone       0.00      0.00      0.00         0
                 water       0.79      0.67      0.72       548
                  food       0.81      0.77      0.79       944
               shelter       0.69      0.67      0.68       691
              clothing       0.52      0.65      0.58        99
                 money       0.40      

In [156]:
print("\t\tAccuracy Score: ")
print("\t\t---------------")
for i in range(Y_test.shape[1]):
    print('%25s : %.2f' %(classes[i], accuracy_score(Y_test.values[:,i], Y_pred[:,i])))
overall_accuracy = (Y_pred == Y_test).mean().mean()
print("\t\t---------------")
print('Average Overall Accuracy : {0:.2f}% \n'.format(overall_accuracy*100))

		Accuracy Score: 
		---------------
                  related : 0.83
                  request : 0.90
                    offer : 1.00
              aid_related : 0.77
             medical_help : 0.92
         medical_products : 0.95
        search_and_rescue : 0.97
                 security : 0.98
                 military : 0.97
              child_alone : 1.00
                    water : 0.96
                     food : 0.95
                  shelter : 0.94
                 clothing : 0.99
                    money : 0.98
           missing_people : 0.99
                 refugees : 0.97
                    death : 0.97
                other_aid : 0.84
   infrastructure_related : 0.92
                transport : 0.95
                buildings : 0.95
              electricity : 0.98
                    tools : 0.99
                hospitals : 0.99
                    shops : 1.00
              aid_centers : 0.99
     other_infrastructure : 0.95
          weather_related : 0.88
      

In [ ]:
#print(classification_report(Y_test.iloc[:,0:].values, np.array([x[0:] for x in Y_pred]), target_names=classes))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle

In [ ]:
pickle.dump(cv, open('model.p', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.